<a href="https://colab.research.google.com/github/averynoah2007/C2C-Group-Project/blob/main/Lesson_3_SQL_to_CSV_to_Insights_FollowAlong.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Analysis & Management — Lesson 3  
## From SQL Results → Insights (CSV + Analysis Workflow)

**Today’s Goal:** Pull data with SQL → export it as a CSV → learn how analysts turn data into insights.

---

### How to use this notebook
- Read the instructions in each section
- Run each code cell (▶️)
- When you see **✅ YOUR TURN**, complete the TODOs
- Use the **Hints** (expand/collapse) for support *without giving exact answers away*

---

### Table of Contents
1. Setup (create a practice database)  
2. Do Now (predict SQL query results)  
3. Section 1 — SQL result sets in Python (tuples → dictionaries)  
4. Section 2 — Export results to CSV + analyze in Google Sheets  
5. Section 3 — Analyze CSV in Python (no Pandas)  
6. Wrap / Preview — Why Pandas DataFrames are the upgrade  


---
# 1) Setup — Create a Practice Database
We will use **SQLite** (a lightweight database that runs inside this notebook).

Run the cell below to create a small practice dataset.


In [ ]:
# ✅ Run this cell first!
import sqlite3
import csv
from collections import defaultdict

# Create an in-memory SQLite database (no installation needed)
conn = sqlite3.connect(":memory:")
cur = conn.cursor()

# Create a simple "students" table for practice
cur.execute("""
CREATE TABLE students (
    student_id INTEGER PRIMARY KEY,
    name TEXT,
    major TEXT,
    gpa REAL,
    absences INTEGER,
    credits INTEGER
);
""")

# Insert sample data (small + realistic)
students_data = [
    (1, "Ava",      "CS",       3.6, 2, 12),
    (2, "Noah",     "Business", 3.1, 5, 10),
    (3, "Mia",      "CS",       3.9, 1, 14),
    (4, "Liam",     "Biology",  2.7, 7, 9),
    (5, "Sophia",   "CS",       3.2, 3, 11),
    (6, "Ethan",    "Business", 2.9, 4, 12),
    (7, "Isabella", "Biology",  3.4, 2, 13),
    (8, "James",    "Art",      3.0, 6, 8),
    (9, "Amelia",   "Business", 3.7, 1, 15),
    (10,"Lucas",    "Art",      2.5, 8, 7),
    (11,"Charlotte","CS",       3.8, 0, 16),
    (12,"Benjamin", "Biology",  3.1, 3, 10),
]

cur.executemany(
    "INSERT INTO students(student_id, name, major, gpa, absences, credits) VALUES (?, ?, ?, ?, ?, ?);",
    students_data
)
conn.commit()

print("✅ Database created!")
print("✅ Table: students")
print("✅ Rows inserted:", len(students_data))


✅ Database created!
✅ Table: students
✅ Rows inserted: 12


---
# 2) Do Now — Predict the SQL Results (7 minutes)

**Before you run any code below:**
Read Query A and Query B, and write down what you *think* they will return.

✅ **Do NOT run them yet — predict first.**

<details>
<summary><b>Hint (what to look for)</b></summary>

- What columns are being selected?
- Is there a `WHERE` filter?
- Is there an `ORDER BY`?
- Are we grouping or counting?
</details>


### Query A (Prediction)
What do you think this query does?

```sql
SELECT name, gpa, major
FROM students
WHERE gpa >= 3.5
ORDER BY gpa DESC;
```


In [ ]:
# ✅ YOUR TURN: Write your prediction (as comments)
# Prediction for Query A:


### Query B (Prediction)
What do you think this query does?

```sql
SELECT major, COUNT(*) AS num_students
FROM students
GROUP BY major
ORDER BY num_students DESC;
```


In [ ]:
# ✅ YOUR TURN: Write your prediction (as comments)
# Prediction for Query B:


## Now run the queries and compare!
After you’ve written your predictions, run the cells below.


In [ ]:
# ✅ Run Query A (returns tuples)
query_a = """
SELECT name, gpa, major
FROM students
WHERE gpa >= 3.5
ORDER BY gpa DESC;
"""
cur.execute(query_a)
results_a = cur.fetchall()

print("Query A results (as tuples):")
results_a


Query A results (as tuples):


[('Mia', 3.9, 'CS'),
 ('Charlotte', 3.8, 'CS'),
 ('Amelia', 3.7, 'Business'),
 ('Ava', 3.6, 'CS')]

In [ ]:
# ✅ Run Query B (returns tuples)
query_b = """
SELECT major, COUNT(*) AS num_students
FROM students
GROUP BY major
ORDER BY num_students DESC;
"""
cur.execute(query_b)
results_b = cur.fetchall()

print("Query B results (as tuples):")
results_b


Query B results (as tuples):


[('CS', 4), ('Business', 3), ('Biology', 3), ('Art', 2)]

---
# 3) Section 1 — SQL Result Sets in Python (15 minutes)

A **result set** is the table of rows + columns returned by a SQL query.

In Python, `fetchall()` returns a **list of tuples**.
Tuples can be hard to read, so analysts often convert rows to **dictionaries**.


## 3.1 Observe the result format (tuples)
Run the cell below. Notice each row is a tuple.


In [ ]:
# ✅ Print Query A rows as tuples
for row in results_a:
    print(row)


## 3.2 Convert tuples → list of dictionaries
Dictionaries keep the **meaning attached** to each value.

<details>
<summary><b>Hint (tuples → dictionaries)</b></summary>

You need 2 things:
1) A list of **column names** (same order as the tuple)
2) A loop using `zip(columns, row)`

Common pattern:
- `dict(zip(columns, row))`
</details>


In [ ]:
# ✅ YOUR TURN: Convert Query A results to dictionaries

columns_a = ["name", "gpa", "major"]
dict_rows_a = []

for row in results_a:
    # TODO: Convert this tuple into a dictionary using columns_a
    # Example target: {"name": "...", "gpa": 0.0, "major": "..."}
    row_dict = None  # <-- replace this
    dict_rows_a.append(row_dict)

# Print the first 3 dictionary rows
print("First 3 dictionary rows:")
for r in dict_rows_a[:3]:
    print(r)


In [ ]:
# ✅ Quick check (auto-check)
# This does NOT show the exact answer — it only checks structure.

assert isinstance(dict_rows_a, list), "dict_rows_a should be a list"
assert len(dict_rows_a) == len(results_a), "dict_rows_a should have the same number of rows as results_a"
assert all(isinstance(x, dict) for x in dict_rows_a), "Each row should be a dictionary"
assert all(set(x.keys()) == {"name","gpa","major"} for x in dict_rows_a), "Each dict should have keys: name, gpa, major"

print("✅ Looks good! You converted tuples → dictionaries.")


---
# 4) Section 2 — Export to CSV + Google Sheets Mini‑Lab (18 minutes)

CSV is one of the most common formats for sharing data across tools.

We’ll export the **full table** to:
**`lesson3_students_export.csv`**


<details>
<summary><b>Hint (what a CSV needs)</b></summary>

- A header row (column names)
- One row per record
- Values separated by commas

In Python, you can use `csv.writer`.
</details>


In [ ]:
# ✅ Run this cell to export the FULL table to CSV

export_query = "SELECT student_id, name, major, gpa, absences, credits FROM students;"
cur.execute(export_query)

rows = cur.fetchall()
headers = [desc[0] for desc in cur.description]  # column names

csv_filename = "lesson3_students_export.csv"

with open(csv_filename, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(headers)   # header row
    writer.writerows(rows)     # data rows

print("✅ Export complete:", csv_filename)
print("Headers:", headers)
print("Rows written:", len(rows))


In [ ]:
# ✅ If you're using Google Colab, this will download the CSV.
# If you're not using Colab, ignore this cell.

try:
    from google.colab import files
    files.download("lesson3_students_export.csv")
except Exception:
    print("Not running in Colab (that's okay).")


## 4.2 Google Sheets Mini‑Lab (do this outside the notebook)

**Open Google Sheets** and import your CSV.

✅ Your mini‑lab tasks:
1. Add a filter to the header row
2. Calculate an average (example: GPA)
3. Find a max value (example: absences)
4. Create 1 chart (example: count of students by major)
5. Write **one insight** (1–2 sentences)

<details>
<summary><b>Hint (where to import CSV)</b></summary>

Google Sheets → File → Import → Upload → select the CSV.
</details>
<details>
<summary><b>Hint (formula ideas)</b></summary>

- `=AVERAGE(D2:D13)` (example range)
- `=MAX(E2:E13)`

*Your range may be different.*
</details>


In [ ]:
## ✅ YOUR TURN: Type your one insight here (1–2 sentences)
# Insight:


---
# 5) Section 3 — Analyze the CSV Back in Python (15 minutes)

Now we’ll do analysis in Python **without Pandas**.
This helps you understand what DataFrames simplify in Lesson 4.


## 5.1 Read the CSV file into Python
We’ll load the CSV back in using `csv.DictReader`.


In [ ]:
# ✅ Read the CSV back into Python

csv_rows = []
with open("lesson3_students_export.csv", "r") as f:
    reader = csv.DictReader(f)
    for row in reader:
        csv_rows.append(row)

print("✅ Rows loaded:", len(csv_rows))
print("Example row:")
csv_rows[0]


## 5.2 Convert numeric columns
CSV imports everything as text. Convert these:
- `student_id` → int
- `gpa` → float
- `absences` → int
- `credits` → int

<details>
<summary><b>Hint (type conversion)</b></summary>

Example pattern:
- `row['absences'] = int(row['absences'])`
</details>


In [ ]:
# ✅ YOUR TURN: Convert numeric columns in-place

for row in csv_rows:
    # TODO: convert each numeric column
    pass

print("Types after conversion:")
print(type(csv_rows[0]["student_id"]), type(csv_rows[0]["gpa"]), type(csv_rows[0]["absences"]), type(csv_rows[0]["credits"]))


In [ ]:
# ✅ Auto-check (structure + types)
required_keys = {"student_id", "name", "major", "gpa", "absences", "credits"}

assert len(csv_rows) > 0, "csv_rows should not be empty"
assert all(set(r.keys()) == required_keys for r in csv_rows), "Each row should have the correct columns"
assert isinstance(csv_rows[0]["student_id"], int), "student_id should be int"
assert isinstance(csv_rows[0]["gpa"], float), "gpa should be float"
assert isinstance(csv_rows[0]["absences"], int), "absences should be int"
assert isinstance(csv_rows[0]["credits"], int), "credits should be int"

print("✅ Numeric columns converted correctly.")


## 5.3 Basic summary stats
Compute:
- total number of rows
- average GPA
- min GPA and max GPA
- top 3 GPAs

<details>
<summary><b>Hint (average GPA)</b></summary>

Steps:
1) Make a list of GPAs
2) `sum(gpas) / len(gpas)`
</details>
<details>
<summary><b>Hint (top 3 GPAs)</b></summary>

Sort by GPA descending, then take the first 3.
</details>


In [ ]:
# ✅ YOUR TURN: Summary stats

# Total rows
total_rows = None  # TODO

# GPA list
gpas = None        # TODO

avg_gpa = None     # TODO
min_gpa = None     # TODO
max_gpa = None     # TODO

print("Total rows:", total_rows)
print("Average GPA:", avg_gpa)
print("Min GPA:", min_gpa)
print("Max GPA:", max_gpa)

# Top 3 GPAs (print name + gpa)
top_3 = None  # TODO (list of 3 items)
print("\nTop 3 GPAs:")
for item in top_3:
    print(item)


## 5.4 Grouping (count students by major)
Goal: create something like:
```python
{'CS': 4, 'Business': 3, ...}
```

<details>
<summary><b>Hint (group counts)</b></summary>

Loop through rows and update a dictionary:
- `counts[major] += 1`

You can use `defaultdict(int)`.
</details>


In [ ]:
# ✅ YOUR TURN: Count students by major

major_counts = None  # TODO

print("Students per major:")
print(major_counts)


---
# 6) Wrap / Preview — Why DataFrames are the upgrade

Today you did a real workflow:
1) SQL → pull data
2) Reshape results (tuples → dictionaries)
3) Export to CSV
4) Analyze in Sheets + Python

**Next lesson:** Pandas DataFrames make analysis faster and require less manual code.


## Optional Preview (Pandas)
This is optional. If Pandas is available, you’ll see how short the analysis becomes.


In [ ]:
# ✅ Optional: Pandas preview (skip if it errors)
try:
    import pandas as pd
    df = pd.DataFrame(csv_rows)
    display(df.head())
    print("\nCount by major:")
    display(df["major"].value_counts())
except Exception:
    print("Pandas not available here (that's okay). We'll learn it in Lesson 4.")


---
# Exit Ticket (Reflection)
Answer these on Canvas (or in class):

1) What is a SQL result set?
2) Why is exporting to CSV useful?
3) Which structure is easier to analyze than tuples?
4) Why do we learn DataFrames next?
